### Use case 

Create a Customer service pre sales chatbot for a fictional company called AI Enchanted Innovations.

Users will come to the website and ask questions about certain products.

If the product is available in the knowledge base, the bot will respond with relevant information.

If the product is not available, the bot will respond with 'We do not sell such product'



### Knowledge Base Data available

We will create a file called crib_description.txt.  We will store the product description.


### Libraries used 

gradio to create Chatbot UIs

langchain to load the knowledge base, chunk it and store in vector store.

FAISS for vector store and search

open AI for embedding and chat responses



### Install Gradio

In [ ]:
#!pip install --upgrade gradio

### Install Langchain

In [ ]:
#!pip install langchain==0.1.0
#!pip install langchain_openai
#!pip install openai==1.7.1


### Loading knowledgebase Documents

In [79]:
from langchain_community.document_loaders import TextLoader

text_loader = TextLoader("crib_description.txt")
knowedge_base_documents = text_loader.load()
knowedge_base_documents


[Document(page_content='Welcome to "AI Enchanted Innovations," where the future of child care meets cutting-edge technology.\n\n\n\n1. The TransformEase AI Convertible Crib\nDescription: Our TransformEase AI Convertible Crib is the ultimate sleep solution that grows with your child. Engineered with AI technology to soothe your baby with gentle rocking motions and serene lullabies, it seamlessly transitions from crib to toddler bed.\nDimensions: 54" L x 29" W x 34" H.\nWeight: 50 lbs.\nPrice: $699.\n\n\n\n\n2. The SerenityNest AI Baby Bassinet\nDescription: The SerenityNest AI Baby Bassinet cradles your newborn in comfort with smart temperature regulation and motion-sensing night lights. Its compact design and whisper-quiet operation ensure a peaceful slumber.\nDimensions: 33" L x 17" W x 39" H.\nWeight: 18 lbs.\nPrice: $399.\n\n\n\n3. The HiGrow AI High Chair\nDescription: Introducing the HiGrow AI High Chair – a smart dining solution that adapts to your child\'s growth. Equipped with 

In [80]:
from langchain.text_splitter import CharacterTextSplitter


text_splitter_definition = CharacterTextSplitter(chunk_size=400)

documents_after_splitting = text_splitter_definition.split_documents(knowedge_base_documents)
print (len(documents_after_splitting))
documents_after_splitting


5


[Document(page_content='Welcome to "AI Enchanted Innovations," where the future of child care meets cutting-edge technology.', metadata={'source': 'crib_description.txt'}),
 Document(page_content='1. The TransformEase AI Convertible Crib\nDescription: Our TransformEase AI Convertible Crib is the ultimate sleep solution that grows with your child. Engineered with AI technology to soothe your baby with gentle rocking motions and serene lullabies, it seamlessly transitions from crib to toddler bed.\nDimensions: 54" L x 29" W x 34" H.\nWeight: 50 lbs.\nPrice: $699.', metadata={'source': 'crib_description.txt'}),
 Document(page_content='2. The SerenityNest AI Baby Bassinet\nDescription: The SerenityNest AI Baby Bassinet cradles your newborn in comfort with smart temperature regulation and motion-sensing night lights. Its compact design and whisper-quiet operation ensure a peaceful slumber.\nDimensions: 33" L x 17" W x 39" H.\nWeight: 18 lbs.\nPrice: $399.', metadata={'source': 'crib_descrip

#### Embed Documents and store in Vector Store

In [81]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("open_ai_secret_key"))
vector_db = FAISS.from_documents(documents_after_splitting, openai_embeddings)

### Sample queries 

In [82]:
query = """describe the crib you have"""
relevant_documents = vector_db.similarity_search_with_score(query)
relevant_document_text = relevant_documents
relevant_document_text
relevant_document_text[0]


(Document(page_content='1. The TransformEase AI Convertible Crib\nDescription: Our TransformEase AI Convertible Crib is the ultimate sleep solution that grows with your child. Engineered with AI technology to soothe your baby with gentle rocking motions and serene lullabies, it seamlessly transitions from crib to toddler bed.\nDimensions: 54" L x 29" W x 34" H.\nWeight: 50 lbs.\nPrice: $699.', metadata={'source': 'crib_description.txt'}),
 0.3756026)

In [84]:
query = """describe the computer desk you have"""
relevant_documents = vector_db.similarity_search_with_score(query)
relevant_document_text = relevant_documents
relevant_document_text[0]

(Document(page_content='3. The HiGrow AI High Chair\nDescription: Introducing the HiGrow AI High Chair – a smart dining solution that adapts to your child\'s growth. Equipped with spill-proof AI sensors, it facilitates tidy eating habits and offers interactive learning activities for mealtime fun.\nDimensions: 22" L x 21" W x 37" H.\nWeight: 15 lbs.\nPrice: $299.', metadata={'source': 'crib_description.txt'}),
 0.5302745)

### Retreive similar content from Vector db

In [108]:
def get_relevant_content(query):
    relevant_documents = vector_db.similarity_search_with_score(query)
    relevant_document_text = relevant_documents[0][0].page_content
    relevant_document_score = relevant_documents[0][1]
    #print (relevant_document_text)
    #print (relevant_document_score)
    if relevant_document_score < 0.40:
        return relevant_document_text
    else:
        return "Sorry!, We do not sell such product"
    #relevant_content = relevant_document_text[0].page_content
    #return relevant_content

get_relevant_content(query)

'Sorry!, We do not sell such product'

In [106]:
import gradio as gr
import os

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatOpenAI(openai_api_key=os.getenv("open_ai_secret_key"))

def chat_with_customer(message, history):
    chat_history = []
    for human_message, ai_message in history:
        chat_history.append(HumanMessage(content=human_message))
        chat_history.append(AIMessage(content=ai_message))
    relevant_document_text = get_relevant_content(message)
    #print ('relevant_document_text',relevant_document_text)
    
    augmented_query = message + "\nalso use below text as reference before answering. \n" + relevant_document_text
    if relevant_document_text == 'Sorry!, We do not sell such product':
        chat_history = []
        llm_response = relevant_document_text
    else:
        chat_history.append(HumanMessage(content=augmented_query))
        #print ('chat_history',chat_history)
        llm_response = chat.invoke(chat_history).content
    return llm_response
    
gr.ChatInterface(chat_with_customer).launch()

Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


1. The TransformEase AI Convertible Crib
Description: Our TransformEase AI Convertible Crib is the ultimate sleep solution that grows with your child. Engineered with AI technology to soothe your baby with gentle rocking motions and serene lullabies, it seamlessly transitions from crib to toddler bed.
Dimensions: 54" L x 29" W x 34" H.
Weight: 50 lbs.
Price: $699.
0.3756026
3. The HiGrow AI High Chair
Description: Introducing the HiGrow AI High Chair – a smart dining solution that adapts to your child's growth. Equipped with spill-proof AI sensors, it facilitates tidy eating habits and offers interactive learning activities for mealtime fun.
Dimensions: 22" L x 21" W x 37" H.
Weight: 15 lbs.
Price: $299.
0.5302745
1. The TransformEase AI Convertible Crib
Description: Our TransformEase AI Convertible Crib is the ultimate sleep solution that grows with your child. Engineered with AI technology to soothe your baby with gentle rocking motions and serene lullabies, it seamlessly transitions